In [1]:
import datetime
# Loading packages

import emoji
import pandas as pd
import re
import nltk
import spacy
nltk.download('wordnet')
from polyglot.detect import Detector

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/franciscorfafonso/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# Loading the dataset and visualizing summary statistics
df = pd.read_excel('Tweets_Working File.xlsx', sheet_name='Tweets_py', engine='openpyxl')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61423 entries, 0 to 61422
Data columns (total 15 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   tweet_id                             61423 non-null  int64 
 1   text                                 61423 non-null  object
 2   author_id                            61423 non-null  int64 
 3   created_at                           61423 non-null  object
 4   public_metrics.retweet_count         61423 non-null  int64 
 5   public_metrics.reply_count           61423 non-null  int64 
 6   public_metrics.like_count            61423 non-null  int64 
 7   public_metrics.quote_count           61423 non-null  int64 
 8   user.id                              61423 non-null  int64 
 9   user.username                        61423 non-null  object
 10  user.created_at                      61423 non-null  object
 11  user.public_metrics.followers_count  6142

In [4]:
# Text preprocessing
def textPreProcess(rawText, removeHTML=True, charsToRemove = r'\?|\.|\!|\;|\.|\"|\,|\(|\)|\&|\:|\-', removeNumbers=True, removeLineBreaks=True, specialCharsToRemove = r'[^\x00-\xfd]', convertToLower=True, removeConsecutiveSpaces=True):
    if type(rawText) != str:
        rawText.astype("string")
    procText = rawText


    #remove ampersand

    procText = re.sub("&amp;","", procText)

    # Remove http
    if removeHTML:
        procText = re.sub(r'http\S+', '', procText)

    # Remove amp

    # Remove mentions
    procText = re.sub(r'@\w+', '', procText)

    #replace emoji by their meaning
    # iterate through the characters in the string
    #procText = emoji.demojize(procText)

    #Fallback Remove emojis
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    procText = emoji_pattern.sub(r'', procText)

    # Remove punctuation and other special characters
    if len(charsToRemove)>0:
        procText = re.sub(charsToRemove,' ',procText)

    # Remove numbers
    if removeNumbers:
        procText = re.sub(r'\d+',' ',procText)

    # Remove line breaks
    if removeLineBreaks:
        procText = procText.replace('\n',' ').replace('\r', '')

    # Remove special characters
    if len(specialCharsToRemove)>0:
        procText = re.sub(specialCharsToRemove,' ',procText)

    # Normalize to lower case
    if convertToLower:
        procText = procText.lower()

        # Replace multiple consecutive spaces with just one space
    if removeConsecutiveSpaces:
        procText = re.sub(' +', ' ', procText)

    return procText


def is_one_word(text):
    """
    Returns True if the input text contains only one word, False otherwise.
    """
    words = text.split()
    return len(words)

def have_emoji (text):
    return emoji.emoji_count(text) > 0

In [5]:
df['PreProcessedText'] = df['text'].apply(textPreProcess)
df['Length'] = df['PreProcessedText'].apply(is_one_word)
#df['Emoji'] = df['text'].apply(have_emoji)

In [6]:
# Remove rows with empty text or only with one word
df.PreProcessedText = df.PreProcessedText.str.strip()
df = df[df.PreProcessedText != '']
df = df[df.Length > 1]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58447 entries, 0 to 61422
Data columns (total 17 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   tweet_id                             58447 non-null  int64 
 1   text                                 58447 non-null  object
 2   author_id                            58447 non-null  int64 
 3   created_at                           58447 non-null  object
 4   public_metrics.retweet_count         58447 non-null  int64 
 5   public_metrics.reply_count           58447 non-null  int64 
 6   public_metrics.like_count            58447 non-null  int64 
 7   public_metrics.quote_count           58447 non-null  int64 
 8   user.id                              58447 non-null  int64 
 9   user.username                        58447 non-null  object
 10  user.created_at                      58447 non-null  object
 11  user.public_metrics.followers_count  5844

In [7]:
df.head(10)

,tweet_id,text,author_id,created_at,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,user.id,user.username,user.created_at,user.public_metrics.followers_count,user.public_metrics.following_count,user.public_metrics.tweet_count,user.public_metrics.listed_count,PreProcessedText,Length
0,1589769441016877056,Deeply honored to be standing in the historic ...,913415985591439362,2022-11-07T23:59:05.000Z,37,18,144,2,913415985591439362,GenDonBolduc,2017-09-28T14:51:56.000Z,25024,4902,10495,145,deeply honored to be standing in the historic ...,37
2,1589767809382617089,🚌BUS TOUR UPDATE🚌\n\nThere’s something so amaz...,3299813970,2022-11-07T23:52:36.000Z,163,65,796,8,3299813970,SmileyForWA,2015-07-28T20:41:43.000Z,35699,1786,4928,144,bus tour update there s something so amazing a...,51
3,1589766458959335424,"North Carolina, the choice is clear.\n\nI will...",2149128326,2022-11-07T23:47:14.000Z,742,47,1704,26,2149128326,CheriBeasleyNC,2013-10-22T14:20:35.000Z,63480,504,2619,401,north carolina the choice is clear i will figh...,15
4,1589766149310656512,"Let's bring this race home, Youngstown! https:...",466532637,2022-11-07T23:46:01.000Z,155,46,575,3,466532637,TimRyan,2012-01-17T14:24:26.000Z,310956,2123,11040,0,let's bring this race home youngstown,6
5,1589764858853359617,In Tulsa talking to voters with @swope4ok and ...,1083019402046513152,2022-11-07T23:40:53.000Z,67,8,194,3,1083019402046513152,VoteKendraOK,2019-01-09T15:15:21.000Z,28165,859,1813,448,in tulsa talking to voters with and there are ...,25
6,1589763186731143171,@dandiekandi @YeetThePolice That account is to...,253622823,2022-11-07T23:34:14.000Z,0,1,0,0,253622823,mchdlny,2011-02-17T16:39:29.000Z,3642,9851,8035,2,that account is totally whackadoodle unhinged ...,26
7,1589763064940765185,"If you have a mail-in ballot, it needs to be r...",3622368202,2022-11-07T23:33:45.000Z,4505,625,10156,115,3622368202,JohnFetterman,2015-09-11T16:59:11.000Z,947574,4942,29747,0,if you have a mail in ballot it needs to be re...,39
8,1589762952961277954,Akron is ready to vote tomorrow! https://t.co/...,466532637,2022-11-07T23:33:19.000Z,239,70,901,8,466532637,TimRyan,2012-01-17T14:24:26.000Z,310956,2123,11040,0,akron is ready to vote tomorrow,6
9,1589760140231995394,"Love to see my staffers Kenneth, Jack, Cody, a...",81191343,2022-11-07T23:22:08.000Z,69,52,767,2,81191343,votetimscott,2009-10-09T20:07:32.000Z,54028,1205,3611,568,love to see my staffers kenneth jack cody and ...,45
10,1589759779773517824,"Thank you for your support, Samuel! We need mo...",1051396218,2022-11-07T23:20:42.000Z,58,14,233,0,1051396218,EvanMcMullin,2012-12-31T23:31:03.000Z,494342,9959,13597,4557,thank you for your support samuel we need more...,45


In [8]:
# Function to check if a tweet is in English
def is_tweet_in_english(text):
    """
    Returns True if the input text is a tweet in English, False otherwise.
    """
    # Detect the language of the text using polyglot's language detector
    try:
        return Detector(text, quiet=True).language.code
    except:
        return None

df['language'] = df['PreProcessedText'].apply(lambda x: is_tweet_in_english(x))

Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to dete

In [9]:
df.head()

,tweet_id,text,author_id,created_at,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,user.id,user.username,user.created_at,user.public_metrics.followers_count,user.public_metrics.following_count,user.public_metrics.tweet_count,user.public_metrics.listed_count,PreProcessedText,Length,language
0,1589769441016877056,Deeply honored to be standing in the historic ...,913415985591439362,2022-11-07T23:59:05.000Z,37,18,144,2,913415985591439362,GenDonBolduc,2017-09-28T14:51:56.000Z,25024,4902,10495,145,deeply honored to be standing in the historic ...,37,en
2,1589767809382617089,🚌BUS TOUR UPDATE🚌\n\nThere’s something so amaz...,3299813970,2022-11-07T23:52:36.000Z,163,65,796,8,3299813970,SmileyForWA,2015-07-28T20:41:43.000Z,35699,1786,4928,144,bus tour update there s something so amazing a...,51,en
3,1589766458959335424,"North Carolina, the choice is clear.\n\nI will...",2149128326,2022-11-07T23:47:14.000Z,742,47,1704,26,2149128326,CheriBeasleyNC,2013-10-22T14:20:35.000Z,63480,504,2619,401,north carolina the choice is clear i will figh...,15,en
4,1589766149310656512,"Let's bring this race home, Youngstown! https:...",466532637,2022-11-07T23:46:01.000Z,155,46,575,3,466532637,TimRyan,2012-01-17T14:24:26.000Z,310956,2123,11040,0,let's bring this race home youngstown,6,en
5,1589764858853359617,In Tulsa talking to voters with @swope4ok and ...,1083019402046513152,2022-11-07T23:40:53.000Z,67,8,194,3,1083019402046513152,VoteKendraOK,2019-01-09T15:15:21.000Z,28165,859,1813,448,in tulsa talking to voters with and there are ...,25,en


In [10]:
# read the Excel file into a Pandas DataFrame
df_cities = pd.read_excel('USAStatesCitiesTownsCounties.xlsx')

# create an empty dictionary to store the results
result_dict = {}

# iterate over the rows of the DataFrame
for index, row in df_cities.iterrows():
    # get the key from one column
    key = row['State short']

    # get the value from other columns
    value1 = row['City alias'].lower()
    value2 = row['City'].lower()
    value3 = row['State full'].lower()
    value4 = row['County'].lower()

    # add the value to the list associated with the key in the dictionary
    if key in result_dict:
        if value1 not in result_dict[key]:
            result_dict[key].append(value1)
        if value2 not in result_dict[key]:
            result_dict[key].append(value2)
        if value3 not in result_dict[key]:
            result_dict[key].append(value3)
        if value4 not in result_dict[key]:
            result_dict[key].append(value4)
    else:
        result_dict[key] = [value1, value2, value3, value4]

# print the resulting dictionary
print(result_dict)

{'NY': ['internal revenue service', 'holtsville', 'new york', 'suffolk', 'fishers island', 'fishers isle', 'gpo', 'macys finance', 'manhattan', 'empire state', 'ny', 'nyc', 'greeley square', 'new york city', 'ny city', 'knickerbocker', 'cooper', 'bowling green', 'wall street', 'trinity', 'church street', 'peter stuyvesant', 'madison square', 'prince', 'chinatown', 'canal street', 'grand central', 'franklin d roosevelt', 'planetarium', 'morningside', 'manhattanville', 'college', 'hamilton grange', 'washington bridge', 'randalls island', 'wards island', 'wards is', 'triborough', 'lincolnton', 'peck slip', 'colonial park', 'fort george', 'citibank', 'roosevelt isl', 'roosevelt island', 'federal reserve', 'merrill lynch', 'jp morgan bank', 'santa claus', 's pole', 'so pole', 'south pole', 'business reply', 'gracie', 'lenox hill', 'midtown', 'bear stearns', 'gpo official mail', 'bank of new york brm', 'bar code church street', 'church street boxes', 'deutsche bank', 'bank of new york', 'eur

In [11]:
# cross each candidate with the short state
df_results = pd.read_excel('SenateResults2022.xlsx', sheet_name='Candidate', engine='openpyxl')
df_results_state = pd.read_excel('SenateResults2022.xlsx', sheet_name='State', engine='openpyxl')
df = pd.merge(df, df_results, left_on='user.username', right_on='handle')
df = pd.merge(df, df_results_state, left_on='state_name', right_on='state_name')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58447 entries, 0 to 58446
Data columns (total 30 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   tweet_id                             58447 non-null  int64  
 1   text                                 58447 non-null  object 
 2   author_id                            58447 non-null  int64  
 3   created_at                           58447 non-null  object 
 4   public_metrics.retweet_count         58447 non-null  int64  
 5   public_metrics.reply_count           58447 non-null  int64  
 6   public_metrics.like_count            58447 non-null  int64  
 7   public_metrics.quote_count           58447 non-null  int64  
 8   user.id                              58447 non-null  int64  
 9   user.username                        58447 non-null  object 
 10  user.created_at                      58447 non-null  object 
 11  user.public_metrics.follower

In [12]:
def check_city(short_state, tweet):
    key_to_search = short_state
    matches = []

    tweet_words = tweet.split()
    # Create a list of every two-word combination
    two_word_combinations = [f"{tweet_words[i]} {tweet_words[i+1]}" for i in range(len(tweet_words)-1)]
    # Combine the two lists
    words_list = two_word_combinations + tweet_words

    for place in result_dict.get(key_to_search, []):
        if place in words_list:
            matches.append(place)
            words_list.remove(place)

    return matches


In [13]:
# Load the English model
nlp = spacy.load('en_core_web_sm')

def check_gpe(row):
    # Initialize an empty list to hold the GPEs
    gpes = []

    # Get the tweet text and list of matches
    tweet = row['PreProcessedText']
    matches = row['check_city']

    # Process the text
    doc = nlp(tweet)

    # Get all named entities that are GPEs
    gpe_ents = [ent.text for ent in doc.ents if ent.label_ == "GPE"]

    # Check each match to see if it's a GPE
    for match in matches:
        if match in gpe_ents:
            gpes.append(match)

    # Return the list of GPEs
    return gpes

In [14]:
df['check_city'] = df[['state_code','PreProcessedText']].apply(lambda x: check_city(*x), axis=1)

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58447 entries, 0 to 58446
Data columns (total 31 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   tweet_id                             58447 non-null  int64  
 1   text                                 58447 non-null  object 
 2   author_id                            58447 non-null  int64  
 3   created_at                           58447 non-null  object 
 4   public_metrics.retweet_count         58447 non-null  int64  
 5   public_metrics.reply_count           58447 non-null  int64  
 6   public_metrics.like_count            58447 non-null  int64  
 7   public_metrics.quote_count           58447 non-null  int64  
 8   user.id                              58447 non-null  int64  
 9   user.username                        58447 non-null  object 
 10  user.created_at                      58447 non-null  object 
 11  user.public_metrics.follower

In [15]:
# Apply the function to each row where 'check_city' is not empty
df['gpe_city'] = df[df['check_city'].str.len() > 0].apply(check_gpe, axis=1)

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58447 entries, 0 to 58446
Data columns (total 32 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   tweet_id                             58447 non-null  int64  
 1   text                                 58447 non-null  object 
 2   author_id                            58447 non-null  int64  
 3   created_at                           58447 non-null  object 
 4   public_metrics.retweet_count         58447 non-null  int64  
 5   public_metrics.reply_count           58447 non-null  int64  
 6   public_metrics.like_count            58447 non-null  int64  
 7   public_metrics.quote_count           58447 non-null  int64  
 8   user.id                              58447 non-null  int64  
 9   user.username                        58447 non-null  object 
 10  user.created_at                      58447 non-null  object 
 11  user.public_metrics.follower

In [17]:
def check_difference(row):
    # Get the lists from 'check_city' and 'gpe_city' columns
    check_city_list = row['check_city']
    gpe_city_list = row['gpe_city']

    # Calculate the difference between the two lists
    difference = list(set(check_city_list) - set(gpe_city_list))

    return difference

# Apply the function to each row in the DataFrame
df['difference'] = df[df['check_city'].str.len() > 0].apply(check_difference, axis=1)

In [18]:
df.head(100)

,tweet_id,text,author_id,created_at,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,user.id,user.username,...,result_pctg,result_votes,position,total_votes_casted_x,state_short,total_votes_casted_y,winner_margin_for_runners_up,check_city,gpe_city,difference
0,1589769441016877056,Deeply honored to be standing in the historic ...,913415985591439362,2022-11-07T23:59:05.000Z,37,18,144,2,913415985591439362,GenDonBolduc,...,0.444346,275928,2,620975,NH,620975,0.090608,"[new hampshire, exeter]",[new hampshire],[exeter]
1,1589758870242500608,Full house tonight! Looking forward to having ...,913415985591439362,2022-11-07T23:17:05.000Z,73,42,270,1,913415985591439362,GenDonBolduc,...,0.444346,275928,2,620975,NH,620975,0.090608,"[new hampshire, exeter]",[new hampshire],[exeter]
2,1589753955592331264,Everyone’s here for the final town hall in Exe...,913415985591439362,2022-11-07T22:57:33.000Z,99,23,569,4,913415985591439362,GenDonBolduc,...,0.444346,275928,2,620975,NH,620975,0.090608,[exeter],[],[exeter]
3,1589723041205886976,Career politicians like @SenatorHassan will sa...,913415985591439362,2022-11-07T20:54:43.000Z,81,51,232,9,913415985591439362,GenDonBolduc,...,0.444346,275928,2,620975,NH,620975,0.090608,[],NaN,NaN
4,1589684478540406784,@sixohfree Thank you!!! 🇺🇸🇺🇸,913415985591439362,2022-11-07T18:21:29.000Z,0,0,2,0,913415985591439362,GenDonBolduc,...,0.444346,275928,2,620975,NH,620975,0.090608,[],NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1587904661327167493,Over 75 town halls and counting 💥\n\nFrom now ...,913415985591439362,2022-11-02T20:29:07.000Z,15,8,48,0,913415985591439362,GenDonBolduc,...,0.444346,275928,2,620975,NH,620975,0.090608,[new hampshire],[new hampshire],[]
96,1587888655750037504,MOMENTUM 💪\n\nGranite Staters are ready for ch...,913415985591439362,2022-11-02T19:25:31.000Z,35,27,104,2,913415985591439362,GenDonBolduc,...,0.444346,275928,2,620975,NH,620975,0.090608,[new hampshire],[new hampshire],[]
97,1587887552580571136,🚨ENDORSEMENT ALERT—I am proud to have the supp...,913415985591439362,2022-11-02T19:21:08.000Z,51,27,174,3,913415985591439362,GenDonBolduc,...,0.444346,275928,2,620975,NH,620975,0.090608,[washington],[washington],[]
98,1587849637213372423,🚨TOMORROW🚨\n\n@mschlapp and @mercedesschlapp j...,913415985591439362,2022-11-02T16:50:28.000Z,16,24,49,0,913415985591439362,GenDonBolduc,...,0.444346,275928,2,620975,NH,620975,0.090608,[loudon],[],[loudon]


In [19]:
df['tweet_id'] = df['tweet_id'].apply(str)
df['author_id'] = df['author_id'].apply(str)
df['user.id'] = df['user.id'].apply(str)
now = datetime.datetime.now()
df.to_excel(f"twitter_wip_{now.strftime('%Y-%m-%d_%H-%M-%S')}.xlsx", sheet_name='tweets_rawdata')